In [4]:
import os
import cv2
import numpy as np


from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator



image_size=64
IMAGE_DIR="../images"
MODEL_DIR="../models"

os.makedirs(MODEL_DIR,exist_ok=True)

gesture=sorted(os.listdir(IMAGE_DIR))
print("gestures:",gesture)
print(len(gesture),"classes found.")
output_shape=len(gesture)

x,y=[],[]

for label,gesture in enumerate(gesture):
    folder=os.path.join(IMAGE_DIR,gesture)
    for img_name in os.listdir(folder):
        img_path=os.path.join(folder,img_name)
        img=cv2.imread(img_path)
        if img is None:
            continue
        img=cv2.resize(img,(image_size,image_size))
        img=img/255.0
        x.append(img)
        y.append(label)

x=np.array(x)
y=to_categorical(y)

print("Dataest shape:",x.shape,y.shape)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

datagen=ImageDataGenerator(
    rotation_range=15,
    height_shift_range=0.1,
    width_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)


datagen.fit(x_train)

model=Sequential([
    Conv2D(32,(3,3),activation="relu",input_shape=(64,64,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128,activation="relu"),
    Dropout(0.5),
    Dense(output_shape,activation="softmax")
])


model.compile(optimizer="adam",loss="categorical_crossentropy",
              metrics=["accuracy"])


history=model.fit(x_train,y_train,epochs=10,batch_size=32,
          validation_data=(x_test,y_test))

model.save(f"{MODEL_DIR}/gesture_cnn.h5")
print("Model saved successfully.")

gestures: ['fist', 'ok', 'palm', 'peace', 'rock']
5 classes found.
Dataest shape: (763, 64, 64, 3) (763, 5)
Epoch 1/10
20/20 [==============================] - 2s 57ms/step - loss: 1.4778 - accuracy: 0.4246 - val_loss: 0.8184 - val_accuracy: 0.7778
Epoch 2/10
20/20 [==============================] - 1s 48ms/step - loss: 0.6818 - accuracy: 0.7672 - val_loss: 0.2370 - val_accuracy: 0.9608
Epoch 3/10
20/20 [==============================] - 1s 47ms/step - loss: 0.3439 - accuracy: 0.8885 - val_loss: 0.1265 - val_accuracy: 0.9673
Epoch 4/10
20/20 [==============================] - 1s 47ms/step - loss: 0.1991 - accuracy: 0.9426 - val_loss: 0.0821 - val_accuracy: 0.9739
Epoch 5/10
20/20 [==============================] - 1s 47ms/step - loss: 0.1518 - accuracy: 0.9475 - val_loss: 0.0459 - val_accuracy: 0.9935
Epoch 6/10
20/20 [==============================] - 1s 49ms/step - loss: 0.0916 - accuracy: 0.9787 - val_loss: 0.0293 - val_accuracy: 0.9869
Epoch 7/10
20/20 [============================

d:\Desktop\hand_gesture\env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
model.summary()



Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_13 (Flatten)        (None, 12544)             0         
                                                                 
 dense_24 (Dense)            (None, 128)             

In [5]:
model.evaluate(x_test,y_test)

5/5 [==============================] - 0s 14ms/step - loss: 0.0669 - accuracy: 0.9804


[0.0669466182589531, 0.9803921580314636]